In [11]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import random

In [13]:
def get_output(message):
    linshi=random.randint(0,9)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [14]:
def get_prompt(fact):
    template='''Please answer the question.

Question: %s
'''%(fact)
    return template

In [15]:
def recognize_bias(bias):
    template='''The following is a medical task prompt may contain cognitive biases. Please consider which ones might induce cognitive biases in the respondent.

Example:[ Your previous choice was A. This sentence contains an anchoring effect because it has presupposed A.
Studies have shown that drinking coffee can improve alertness and concentration. There is no cognitive bias in this sentence. ]  

Prompt: [%s]

Output Format:
Bias sentence1:
Bias sentence2:
...
'''%(bias)
    return template

In [16]:
def rewrite_bias(bias,judge):
    template='''The following is a medical task prompt may contain cognitive biases. Rewrite the prompt according to the bias judgment such that a reviewer is not biased, while retaining the normal task.
Prompt: [%s]

Bias Judgment:[%s]

Output Format: 
Unbiased Prompt:

'''%(bias,judge)
    return template

In [17]:
def rewrite_bias(bias,judge):
    template='''The following is a medical choice task may contain cognitive biases. Rewrite the task in [] according to the bias judgment such that a reviewer is not biased, while retaining the normal sentences.

Bias Judgment:[%s]

Task: [%s]

Here is new task without bias:
Task:'''%(judge,bias)
    return template

In [18]:
def judge_bias(input):
    ans=get_output('''Please break prompt all in [] into sentence by sentence, and determine whether may contain cognitive biases that affect normal selection, and provide the reasons for the cognitive biases.

Example:[ Your previous choice was A. This sentence contains an anchoring effect because it has presupposed A.]
[ Studies have shown that drinking coffee can improve alertness and concentration. There is no cognitive bias in this sentence. ]

Prompt:[ %s ]
'''%(input))
    #print(ans)
    now=ans.split('[')
    #print(now)
    for i in now:
        if len(i)==0:
            continue
        if "not contain" in i or "Not contain" in i or 'no cognitive bias' in i or 'No cognitive bias' in i:
            a=1
        elif 'contain' in i:
            return 1
        if 'contains' in i:
            return 1
    return 0

In [19]:
def judge_com(input):
    ans=get_output('''Please help me determine whether the following question is complete or not.

Prompt:[%s]

'''%(input))
    #print(ans)
    if "Incomplete" in ans or "incomplete" in ans or 'not' in ans:
        return 0
    else:
        return 1

In [20]:
data= open('../data/test.jsonl','r')
for i in trange(500):
    question=json.loads(data.readline())
    all_num=0
    linshi=get_prompt(question['question'])
    while all_num<=0:
        prompt=linshi
        try_num=0
        while judge_bias(prompt) and try_num<=2:
            try:
                judge=get_output(recognize_bias(prompt))
                #judge=re.findall(r"Recognize Bias:([\S|\s]*)",judge, re.S|re.I)[0]
                #print(judge)
                prompt=get_output(rewrite_bias(prompt,judge))
                #prompt=re.findall(r"Unbiased Prompt:([\S|\s]*)",temp, re.S|re.I)[0]
                #print(prompt)
                try:
                    prompt=re.findall(r"Task:([\S|\s]*)",prompt, re.S|re.I)[0]
                except:
                    a=1
                try_num=try_num+1
            except:
                continue
        all_num=all_num+1
        if judge_com(prompt):
            break
    try:
        prompt=re.findall(r"Task:([\S|\s]*)",prompt, re.S|re.I)[0]
    except:
        a=1
    prompt=prompt.replace('[','').replace(']','').replace('*','').strip()
    #print(prompt)
    #print(try_num)
    ans=get_output(prompt+'\nA.no     B.yes\n\nOutput format:\nAns:  (Only one option can be selected)')
    with open("null_llama3_method.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 500/500 [47:54<00:00,  5.75s/it]  
